# DSAP 107

In [1]:
import numpy as np
import pandas as pd 
from datetime import datetime

### 讀取資料

In [6]:
grades = pd.read_csv('../../data/DSAP107 grades.csv')
v_info = pd.read_excel('../../data/DSAP107 videos_sep.xlsx', sheet_name = 'video info')
records = pd.read_excel('../../data/DSAP107 videos_sep.xlsx', sheet_name = 'watching records')
student_id = pd.read_excel('../../data/DSAP107 videos_sep.xlsx', sheet_name = 'student id')

In [10]:
grades

,student_id,Section,Homework 0 (1554),Homework 1 (1475),Homework 2 (1750),Homework 3 (1808),Homework 4 (1859),Homework 5 (1954),Homework 6 (2053),Homework 7 (2129),Homework8 (2232),Quiz (1582),Midterm (2042),Final (2346)
0,Points Possible,NaN,100,100,100,100,100,100,100,100,100,100.0,0,100.0
1,64,107-2 資料結構與進階程式設計,0,0,0,0,0,0,0,0,0,NaN,0,0.0
2,65,107-2 資料結構與進階程式設計,14,0,0,0,0,0,0,0,0,NaN,0,0.0
3,66,107-2 資料結構與進階程式設計,88,39,96,104,69,98,54,54,96,NaN,82,76.0
4,67,107-2 資料結構與進階程式設計,99,95,46,92,32,42,53,97,97,NaN,79,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,164,107-2 資料結構與進階程式設計,100,59,65,88,82,94,98,91,97,NaN,0,88.0
102,165,107-2 資料結構與進階程式設計,91,98,96,110,82,96,81,89,96,NaN,0,55.0
103,166,107-2 資料結構與進階程式設計,100,60,100,115,111,98,98,96,92,NaN,90,76.0
104,167,107-2 資料結構與進階程式設計,91,100,99,112,97,98,120,92,97,NaN,91,54.0


### 其中、期末成績

In [11]:
# 開一個 table 存等一下計算的資料
student_info = pd.DataFrame(data = grades.drop(0)['student_id'])

student_info['midterm'] = grades['Midterm (2042)']
student_info['final'] = grades['Final (2346)']

### 觀看時間、觀看速度、平均觀看次數

In [12]:
### 觀看時間、觀看速度、平均觀看次數


# 計算總觀看時間
total_spent_time = [] # 總觀看時間
total_watch_video_time = [] # 觀看的總影片時間

for i in student_info['student_id']:
    watch_record_i = records[records['student_id'] == int(i)]
    # 抓出有在看的資料
    real_watch_i = watch_record_i[(watch_record_i['end'] > watch_record_i['start']) & (watch_record_i['playback_rate'] != 0)]
    video_time_i = real_watch_i['end'] - real_watch_i['start']# 觀看紀錄 的 影片時間(end - start)
    total_watch_video_time.append(sum(video_time_i))
    total_spent_time.append(sum(video_time_i/real_watch_i['playback_rate']))

student_info['total_watch_time'] = total_spent_time

# 計算撥放速度
total_spent_time = [x if x!= 0 else 1 for x in total_spent_time]
avg_playback_rate = [i/j for (i,j) in zip(total_watch_video_time, total_spent_time)]
student_info['avg_playback_rate'] = avg_playback_rate

# 計算影片平均觀看次數
length_l = list(v_info['meta'])# length of each lecture
length_l = [int(i.split('n=>')[1].split('}')[0]) for i in length_l]
total_length_l = sum(length_l)
student_info['avg_freq'] = student_info['total_watch_time']/(total_length_l/student_info['avg_playback_rate'])

student_info

,student_id,midterm,final,total_watch_time,avg_playback_rate,avg_freq
1,64,0,0.0,13331.983333,1.492651,0.563979
2,65,0,0.0,3572.000000,1.000000,0.101233
3,66,82,76.0,36625.000000,1.000000,1.037976
4,67,79,79.0,18789.383333,1.847426,0.983761
5,68,52,57.0,13719.766667,1.742595,0.677568
...,...,...,...,...,...,...
101,164,0,88.0,28647.200000,1.376958,1.117925
102,165,0,55.0,6125.250000,1.626709,0.282386
103,166,90,76.0,18766.033333,1.279866,0.680686
104,167,91,54.0,17117.583333,1.565057,0.759246


### Save data

In [13]:
student_info.to_excel('../../data/許哲/DSAP107.xlsx')

### Regression

In [23]:
data = student_info[student_info['midterm'] != 0]
data

,student_id,midterm,final,total_watch_time,avg_playback_rate,avg_freq
3,66,82,76.0,36625.000000,1.000000,1.037976
4,67,79,79.0,18789.383333,1.847426,0.983761
5,68,52,57.0,13719.766667,1.742595,0.677568
7,70,71,80.5,31772.600000,1.563360,1.407737
8,71,98,91.5,12522.250000,1.814291,0.643871
...,...,...,...,...,...,...
96,159,52,65.5,25794.850000,1.530577,1.118917
100,163,79,84.0,24626.633333,1.448310,1.010826
103,166,90,76.0,18766.033333,1.279866,0.680686
104,167,91,54.0,17117.583333,1.565057,0.759246


In [28]:
import statsmodels.formula.api as smf
result = smf.ols('midterm ~ avg_playback_rate + total_watch_time+avg_freq', data = data).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                midterm   R-squared:                       0.094
Model:                            OLS   Adj. R-squared:                  0.048
Method:                 Least Squares   F-statistic:                     2.051
Date:                Wed, 27 May 2020   Prob (F-statistic):              0.117
Time:                        00:49:25   Log-Likelihood:                -261.71
No. Observations:                  63   AIC:                             531.4
Df Residuals:                      59   BIC:                             540.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            25.2663     28.07